In [ ]:
import pandas as pd  
import netCDF4 as nc  
import numpy as np  

file_path = "your_file.nc"                       # Replace 'your_file.nc' with the path to your actual NetCDF file

# Open the NetCDF file for reading
with nc.Dataset(file_path, 'r') as dataset:  
    time_var = dataset.variables['time'][:]      # Replace 'time' with the actual variable name for the time in your file
    lat_var = dataset.variables['lat'][:]        # Replace 'lat' with the actual variable name for latitude
    lon_var = dataset.variables['lon'][:]        # Replace 'lon' with the actual variable name for longitude
    data_var = dataset.variables['variable'][:]  # Replace 'variable' with the actual variable name for data

time_2d, lat_2d, lon_2d = np.meshgrid(time_var, lat_var, lon_var, indexing='ij')  
data_1d = data_var.flatten()  
df = pd.DataFrame({                              # Replace 'df' with the name you want to initialize for your DataFrame
    'time': time_2d.flatten(),  
    'lat': lat_2d.flatten(),   
    'lon': lon_2d.flatten(),    
    'variable': data_1d        
})
df  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ruptures as rpt

# Replace 'df' with the actual DataFrame name in your analysis
df['datetime'] = pd.to_datetime(df['time'])  
df.set_index('datetime', inplace=True)  
df['Year'] = df.index.year + df.index.dayofyear / 365  
df['Month'] = df['Year'] * 12  

# Detect change points in the average data using the PELT method with an RBF kernel

signal = df['variable name'].values       ### Replace 'variable name' with your variable name ###
algo = rpt.Pelt(model="rbf").fit(signal)  # Apply PELT method for change point detection using the Radial Basis Function (RBF) model
change_points = algo.predict(pen=6)       ### Adjust the penalty value to control the sensitivity of change point detection ###

plt.figure(figsize=(14, 8))               ### Set the figure size ###

# Replace the 'your variable' with your variable name
plt.plot(df['Month'], df['your variable'], label='Variable Name', color='#FF5733', linewidth=2)  # Change the color code as per your requirment

# Replace the 'min', 'max' and 'your variable' with your variable maximum and minimum column name and variable column name 
plt.fill_between(df['Month'], df['min'], df['max'], color='lightgray', alpha=0.7, label='Range of your variable')  

# Plot vertical lines at detected change points
for cp in change_points[:-1]:  
    plt.axvline(df['Month'].iloc[cp], color='purple', linestyle='--', label='Change Point' if cp == change_points[0] else "")  
trend_colors = ['blue', 'green', 'orange', 'purple']  

# Calculate and plot linear trend lines for each segment between change points
time_values = df['Month'].values.reshape(-1, 1)  
previous_cp = 0  
slopes = []  
handles = []  
labels = []  

# Loop through each change point to create segments
for idx, cp in enumerate(change_points):  
    segment = slice(previous_cp, cp) 
    model_segment = LinearRegression().fit(time_values[segment], df['average'].values[segment])  # Fit a linear regression model to the segment
    slope = model_segment.coef_[0]  
    slopes.append(slope)  
    trend_label = f'Trend {idx + 1}'  
    handles.append(plt.Line2D([0], [0], linestyle='--', color=trend_colors[idx % len(trend_colors)], label=trend_label))  
    plt.plot(df['Month'].values[segment], model_segment.predict(time_values[segment]), linestyle='--', color=trend_colors[idx % len(trend_colors)], label=trend_label)  
    previous_cp = cp  

# Print slopes of each segment
for i, slope in enumerate(slopes):  
    print(f"Slope of Trend {i + 1}: {slope:.3f} mm/month")  

# Add a legend to the plot with 3 rows per column
handles, labels = plt.gca().get_legend_handles_labels()  

# Arrange handles and labels for a 2-column layout with 3 rows per column
plt.legend(handles=handles, labels=labels, loc='upper left', bbox_to_anchor=(0.0, 1.02), 
           frameon=False, fancybox=False, shadow=False, prop={'size': 28, 'weight': 'bold'}, 
           ncol=2, handlelength=2.5, handletextpad=1, columnspacing=2) 

# Label x and y axes
plt.xlabel('Year', fontsize=28, fontweight='bold', color='black')  
plt.ylabel('Variable Name', fontsize=28, fontweight='bold', color='black')  

# Customize x and y ticks
plt.xticks(fontsize=28, fontweight='bold', color='black')  
plt.yticks(fontsize=28, fontweight='bold', color='black')  


specific_years = [1999, 2002, 2005, 2008, 2011, 2014, 2017, 2020, 2023]   ### Set specific years as x-ticks ###
specific_months = [year * 12 for year in specific_years] 
plt.xticks(specific_months, specific_years)  

# Set the limits for the x and y axes
plt.xlim(df['Month'].min(), df['Month'].max())  
plt.ylim(0, 210)  

# Enhance grid and spines for a cleaner appearance
plt.grid(True, which='both', linestyle=':', linewidth=0.5, color='gray') 
plt.gca().spines['top'].set_color('black') 
plt.gca().spines['right'].set_color('black') 
plt.gca().spines['bottom'].set_color('black')  
plt.gca().spines['left'].set_color('black')  

# Save the plot as a TIFF file
plt.savefig('your_filename.tiff', dpi=2000, bbox_inches='tight', facecolor='white')  # Replace 'your_filename.tiff' with the desired filename when saving the plot

# Show the plot
plt.show()  